## Imports

In [1]:
from fastapi import FastAPI, Request
from fastapi.responses import FileResponse, Response
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional
from openai import OpenAI
from dotenv import load_dotenv
from riotwatcher import LolWatcher, ApiError
import httpx
import asyncio
import logging
import os
import requests
import random
import arrow

# Load Cassiopeia
import cassiopeia as cass
from cassiopeia import Summoner
from cassiopeia import Locales
from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Queue, Patch

from sortedcontainers import SortedList


RIOT_API_KEY = "RGAPI-de85bb37-76e5-6890-9a94-fcf7b1bf14a6"

load_dotenv()
cass.set_riot_api_key(os.getenv("RIOT_API_KEY"))

## Locales

In [2]:
def get_locales():
    locales = cass.get_locales(region="NA")
    print(locales)
    for locale in locales:
        print(locale)
    print(locales)


if __name__ == "__main__":
    get_locales()

[...]
Making call: https://ddragon.leagueoflegends.com/cdn/languages.json
en_US
cs_CZ
de_DE
el_GR
en_AU
en_GB
en_PH
en_SG
es_AR
es_ES
es_MX
fr_FR
hu_HU
it_IT
ja_JP
ko_KR
pl_PL
pt_BR
ro_RO
ru_RU
th_TH
tr_TR
vi_VN
zh_CN
zh_MY
zh_TW
['en_US', 'cs_CZ', 'de_DE', 'el_GR', 'en_AU', 'en_GB', 'en_PH', 'en_SG', 'es_AR', 'es_ES', 'es_MX', 'fr_FR', 'hu_HU', 'it_IT', 'ja_JP', 'ko_KR', 'pl_PL', 'pt_BR', 'ro_RO', 'ru_RU', 'th_TH', 'tr_TR', 'vi_VN', 'zh_CN', 'zh_MY', 'zh_TW']


## Summoner

In [3]:
def print_summoner(name: str, region: str):
    summoner = Summoner(name=name, region=region)
    print("Name:", summoner.name)
    print("puuid:", summoner.id)
    print("Account ID:", summoner.account_id)
    print("Level:", summoner.level)
    print("Revision date:", summoner.revision_date)
    print("Profile icon ID:", summoner.profile_icon.id)
    print("Profile icon name:", summoner.profile_icon.name)
    print("Profile icon URL:", summoner.profile_icon.url)
    print("Profile icon image:", summoner.profile_icon.image)


if __name__ == "__main__":
    print_summoner("gameb0x", "NA")

Name: gameb0x
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/gameb0x
puuid: 7SKyjHwyyrKgwvMM4tQaP72Hjwb8hveVobJfmD1aydG-2TU
Account ID: YUbdgjG3omxSZulZg3S4pUWo9fB6ZAfF529YbKLUsoeL9w
Level: 78
Revision date: 2023-11-09T13:00:42+00:00
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Profile icon ID: 6330
Profile icon name: None
Profile icon URL: https://ddragon.leagueoflegends.com/cdn/13.23.1/img/profileicon/6330.png
Making call: https://ddragon.leagueoflegends.com/cdn/13.23.1/img/profileicon/6330.png
Profile icon image: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=300x300 at 0x11548B250>


## Match Collection

In [4]:
def filter_match_history(summoner, patch):
    end_time = patch.end
    if end_time is None:
        end_time = arrow.now()
    match_history = MatchHistory(
        summoner=summoner,
        queues={Queue.aram},
        begin_time=patch.start,
        end_time=end_time,
    )
    return match_history


def collect_matches():
    initial_summoner_name = "gameb0x"
    region = "NA"

    summoner = Summoner(name=initial_summoner_name, region=region)
    patch = Patch.from_str("8.9", region=region)

    unpulled_summoner_ids = SortedList([summoner.id])
    pulled_summoner_ids = SortedList()

    unpulled_match_ids = SortedList()
    pulled_match_ids = SortedList()

    while unpulled_summoner_ids:
        # Get a random summoner from our list of unpulled summoners and pull their match history
        new_summoner_id = random.choice(unpulled_summoner_ids)
        new_summoner = Summoner(id=new_summoner_id, region=region)
        matches = filter_match_history(new_summoner, patch)
        unpulled_match_ids.update([match.id for match in matches])
        unpulled_summoner_ids.remove(new_summoner_id)
        pulled_summoner_ids.add(new_summoner_id)

        while unpulled_match_ids:
            # Get a random match from our list of matches
            new_match_id = random.choice(unpulled_match_ids)
            new_match = Match(id=new_match_id, region=region)
            for participant in new_match.participants:
                if (
                    participant.summoner.id not in pulled_summoner_ids
                    and participant.summoner.id not in unpulled_summoner_ids
                ):
                    unpulled_summoner_ids.add(participant.summoner.id)
            # The above lines will trigger the match to load its data by iterating over all the participants.
            # If you have a database in your datapipeline, the match will automatically be stored in it.
            unpulled_match_ids.remove(new_match_id)
            pulled_match_ids.add(new_match_id)


if __name__ == "__main__":
    collect_matches()

Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json


TypeError: MatchHistory.__get_query_from_kwargs__() got an unexpected keyword argument 'summoner'

## Match Statistics

In [5]:
def get_participant_data(participants):
    team_participant_data = []
    for p in participants:
        p_data = {"summoner": p.summoner.name, "champion": p.champion.name, "runes": p.runes.keystone.name,
                    "d_spell": p.summoner_spell_d.name, "f_spell": p.summoner_spell_f.name,
                    "kills": p.stats.kills, "assist": p.stats.assists, "deaths": p.stats.deaths,
                    "kda_ratio": round(p.stats.kda, 2), "damage_dealt": p.stats.total_damage_dealt,
                    "creep_score": p.stats.total_minions_killed, "vision_score": p.stats.vision_score}

        p_items = []
        number_of_item_slots = 6
        for i in range(number_of_item_slots):
            try:
                p_items.append(p.stats.items[i].name)
            except AttributeError:
                p_items.append("None")
        p_data["items"] = p_items
        team_participant_data.append(p_data)
    return team_participant_data


def print_match_history(summoner, num_matches):
    for i, match in enumerate(summoner.match_history().matches[:num_matches], start=1):
        match_num = f"MATCH {i}"
        print(match_num.center(100, "#"))

        blue_team = match.blue_team.participants
        print("BLUE SIDE".center(100, "-"))
        for p_data in get_participant_data(blue_team):
            print(p_data)

        red_team = match.red_team.participants
        print("RED SIDE".center(100, "-"))
        for p_data in get_participant_data(red_team):
            print(p_data)


def main():
    name = "gameb0x"
    server = "NA"
    summoner = cass.get_summoner(name=name, region=server)
    print_match_history(summoner, 5)


if __name__ == "__main__":
    main()

TypeError: MatchHistory.__get_query_from_kwargs__() missing 2 required keyword-only arguments: 'continent' and 'puuid'